In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/catalogues/balona_supp/table1.dat', header=None, delim_whitespace=True, 
                 names=['kic', 'Type', 1,2,3,4,5,6,7],
                 dtype={'kic': str}
                )[['kic', 'Type']]
# df = df[df['Type'] == 'DSCT']
df = df[df.Type.isin(['DSCT', 'DSCT/EB', 'DSCT/EB/ROT', 'DSCT/HRTB', 'DSCT/MAIA',
       'DSCT/MAIA/ROT', 'DSCT/ROT', 'DSCT/SOLR', 'DSCT/SOLR?', 'DSCT/SPB',
       'DSCT?'])]

af = pd.read_csv('../data/catalogues/rev_AF_stars.csv', dtype={'kic': str})[['kic', 'dSct']]
af = af[af['dSct'] > 0]

FileNotFoundError: [Errno 2] No such file or directory: '../data/catalogues/balona_supp/table1.dat'

In [3]:
np.unique(df.Type)

array(['DSCT', 'DSCT/EB', 'DSCT/EB/ROT', 'DSCT/HRTB', 'DSCT/MAIA',
       'DSCT/MAIA/ROT', 'DSCT/ROT', 'DSCT/SOLR', 'DSCT/SOLR?', 'DSCT/SPB',
       'DSCT?'], dtype=object)

In [4]:
df[df.Type.isin(['DSCT', 'DSCT/EB', 'DSCT/EB/ROT', 'DSCT/HRTB', 'DSCT/MAIA',
       'DSCT/MAIA/ROT', 'DSCT/ROT', 'DSCT/SOLR', 'DSCT/SOLR?', 'DSCT/SPB',
       'DSCT?'])]

,kic,Type
0,1026294,DSCT
2,1162150,DSCT
3,1163943,DSCT
4,1294670,DSCT
5,1430590,DSCT
...,...,...
2469,12647070,DSCT/ROT
2470,12688835,DSCT
2472,12736056,DSCT/MAIA
2473,12784394,DSCT


In [8]:
df = df.merge(af, left_on='kic', right_on='kic', how='outer')['kic']

In [14]:
df.values

array(['1026294', '1162150', '1163943', ..., '12520187', '12603159',
       '12691831'], dtype=object)

In [15]:
from astroquery.mast import Observations, Catalogs

result = Catalogs.query_criteria(catalog="Tic", KIC=df.values).to_pandas()

/Users/daniel/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3343: TableReplaceWarning: converted column 'HIP' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/daniel/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3343: TableReplaceWarning: converted column 'numcont' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/daniel/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3343: TableReplaceWarning: converted column 'gaiaqflag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/daniel/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3343: TableReplaceWarning: converted column 'raddflag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
result['KIC']

0        6756386
1        8282338
2        6266750
3        6587551
4        9758615
          ...   
2349     5991936
2350     5480730
2351     5130650
2352     9851822
2353    10936474
Name: KIC, Length: 2354, dtype: int64

In [20]:
result.to_csv('dsct.csv')

In [16]:
# all_kics = df.kic.values
wget_commands = []
for kid in df.kic.values:
    kicid = str(kid).zfill(9)
    wget = "wget -nH --cut-dirs=6 -r -l0 -c -q -N -np -R 'index*' -erobots=off -A 'kplr*_lc_*.tar' https://archive.stsci.edu/pub/kepler/lightcurves/" + kicid[:4] + '/' + kicid + '/'
    wget_commands.append(wget)
    
np.savetxt('kic_hot_stars.sh', wget_commands, delimiter=" ", fmt="%s")